In [28]:
import re
import logging
import numpy as np
import pandas as pd
import multiprocessing

from re import sub
from time import time 
from gensim.models import Word2Vec
from collections import defaultdict
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser
import tweepy as tw
from sklearn.cluster import KMeans

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [29]:
consumer_key= 'GoN90XkTehFV4lNMk1Whk8JYe'
consumer_secret= 'WCpGzX2T67hAF56EU1qdAFo0lO5EoIv7fgrEoNtwa8dvv1fuLz'
access_token= '1417219540904259594-CdqMIpFBK10mF7eY9l2LRBNW1PbD53'
access_token_secret= 'G1P2vYYXB3BeShIG5fvPEgYMFpY5sB0LMfS5mfHk2yLz9'

In [30]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api =tw.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [31]:
def dataframe(text, size = 100, lang = "en"):
    tweets = tw.Cursor(api.search ,q= text, lang=lang , tweet_mode="extended").items(size)
    tweet =[]
    for i in tweets :
        tweet.append(i.full_text)
    df = pd.DataFrame({'tweet': tweet})
    #Removing retweets:
    df = df[~df.tweet.str.contains("RT")] 
    df= df.reset_index(drop=True)
    return df

In [32]:
def preprocess(text):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    text = " ".join(new_text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    text = text.split()

    return text  

In [83]:
df = dataframe("#esg", 100000)
df.head(5)

WARNING - 09:48:18: Rate limit reached. Sleeping for: 31
WARNING - 09:49:39: Rate limit reached. Sleeping for: 855
WARNING - 10:04:47: Rate limit reached. Sleeping for: 853
WARNING - 10:19:54: Rate limit reached. Sleeping for: 852
WARNING - 10:35:00: Rate limit reached. Sleeping for: 851
WARNING - 10:50:03: Rate limit reached. Sleeping for: 853


,tweet
0,The HKTDC hosted a #webinar last month on “Sus...
1,"As the main asset owners in the region, pensio..."
2,We are proud to announce that our Swedish alum...
3,The need for more sustainable investments is g...
4,9/10 The Value Preservation Imperative (Milest...


In [84]:
df.to_csv("100000_tweets_esg.csv", index = False)

In [85]:
df = pd.read_csv("100000_tweets_esg.csv")

In [86]:
df.tweet = df.tweet.apply(lambda x: preprocess(x))

In [87]:
df.head()

,tweet
0,"[the, hktdc, hosted, a, webinar, last, month, ..."
1,"[as, the, main, asset, owners, in, the, region..."
2,"[we, are, proud, to, announce, that, our, swed..."
3,"[the, need, for, more, sustainable, investment..."
4,"[9/10, the, value, preservation, imperative, m..."


In [88]:
sent = [row for row in df.tweet]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]

INFO - 11:21:21: collecting all words and their counts
INFO - 11:21:21: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 11:21:21: collected 113936 word types from a corpus of 213207 words (unigram + bigrams) and 7042 sentences
INFO - 11:21:21: using 113936 counts as vocab in Phrases<0 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 11:21:21: source_vocab length 113936
INFO - 11:21:22: Phraser built with 10070 phrasegrams


In [89]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20)

In [90]:
w2v_model.build_vocab(sentences, progress_per=50000)

INFO - 11:21:32: collecting all words and their counts
INFO - 11:21:32: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 11:21:33: collected 25187 word types from a corpus of 175312 raw words and 7042 sentences
INFO - 11:21:33: Loading a fresh vocabulary
INFO - 11:21:33: effective_min_count=3 retains 7905 unique words (31% of original 25187, drops 17282)
INFO - 11:21:33: effective_min_count=3 leaves 152306 word corpus (86% of original 175312, drops 23006)
INFO - 11:21:33: deleting the raw counts dictionary of 25187 items
INFO - 11:21:33: sample=1e-05 downsamples 5593 most-common words
INFO - 11:21:33: downsampling leaves estimated 39840 word corpus (26.2% of prior 152306)
INFO - 11:21:33: estimated required memory for 7905 words and 100 dimensions: 10276500 bytes
INFO - 11:21:33: resetting layer weights


In [91]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

INFO - 11:21:40: training model with 3 workers on 7905 vocabulary and 100 features, using sg=0 hs=0 sample=1e-05 negative=20 window=4
INFO - 11:21:41: EPOCH 1 - PROGRESS: at 69.13% examples, 26762 words/s, in_qsize 0, out_qsize 0
INFO - 11:21:41: worker thread finished; awaiting finish of 2 more threads
INFO - 11:21:41: worker thread finished; awaiting finish of 1 more threads
INFO - 11:21:41: worker thread finished; awaiting finish of 0 more threads
INFO - 11:21:41: EPOCH - 1 : training on 175312 raw words (39845 effective words) took 1.5s, 26892 effective words/s
INFO - 11:21:42: EPOCH 2 - PROGRESS: at 69.13% examples, 26557 words/s, in_qsize 0, out_qsize 0
INFO - 11:21:43: worker thread finished; awaiting finish of 2 more threads
INFO - 11:21:43: worker thread finished; awaiting finish of 1 more threads
INFO - 11:21:43: worker thread finished; awaiting finish of 0 more threads
INFO - 11:21:43: EPOCH - 2 : training on 175312 raw words (39672 effective words) took 1.5s, 27284 effectiv

(1194281, 5259360)

In [12]:
#w2v_model.save("word2vec_#Tesla.model")
#word_vectors = Word2Vec.load("word2vec_#Tesla.model").wv

In [93]:
word_vectors = w2v_model.wv

In [94]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=w2v_model.wv.vectors.astype('double'))

In [96]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=20, restrict_vocab=None)

[('animal', 0.9999197721481323),
 ('this_episode', 0.9999191761016846),
 ('ready', 0.99991774559021),
 ('ban', 0.9999140501022339),
 ('esgtoday_user', 0.9999138712882996),
 ('flag', 0.9999130964279175),
 ('whc2022_sustainability', 0.9999117851257324),
 ('aligned', 0.999911367893219),
 ('71', 0.9999109506607056),
 ('supported_by', 0.9999109506607056),
 ('secrets', 0.9999107718467712),
 ('u', 0.999909520149231),
 ('leap', 0.9999093413352966),
 ('outlook', 0.9999092817306519),
 ('greenbonds', 0.9999083280563354),
 ('sponsored_by', 0.9999071955680847),
 ('friday', 0.9999070763587952),
 ('our_planet', 0.9999070167541504),
 ('argument', 0.9999069571495056),
 ('mention', 0.9999067783355713)]

In [45]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [25]:
words = pd.DataFrame(word_vectors.index2word)
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [26]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [28]:
words.head(100)

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,tesla,"[0.020728745, 0.19216748, 0.124839455, -0.1515...",2,-1,1.823851,-1.823851
1,user,"[0.013822517, 0.09307468, 0.054610573, -0.0721...",3,-1,3.648602,-3.648602
2,http,"[0.012700567, 0.15408088, 0.09388471, -0.10973...",2,-1,5.472747,-5.472747
3,the,"[0.017991967, 0.18609089, 0.11322693, -0.13930...",2,-1,2.303094,-2.303094
4,a,"[0.01618166, 0.1769216, 0.10947525, -0.1335849...",2,-1,2.500294,-2.500294
...,...,...,...,...,...,...
95,modely,"[0.0057655363, 0.009893008, 0.0067028073, -0.0...",0,-1,14.950323,-14.950323
96,about,"[0.0074896715, 0.084602125, 0.051615532, -0.06...",3,-1,4.299760,-4.299760
97,amp,"[0.0116495695, 0.057826363, 0.033274837, -0.04...",3,-1,32.534349,-32.534349
98,000,"[0.0007470698, 0.018412301, 0.014753984, -0.01...",1,1,12.486751,12.486751
